In [8]:
import os
import re
import subprocess
from collections import defaultdict
import numpy as np
import rasterio
from rasterio.merge import merge
from rasterio.warp import reproject, Resampling
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import glob
import json
from tqdm import tqdm

#import geopandas as gpd
import pandas as pd

%load_ext autoreload
%autoreload 2

import sys
sys.path.append('../')

import utils.basics as bsc 
import utils.plotting as pt
import utils.processing as proc

from src.config_loader import get_config, load_yaml


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [10]:
data = load_yaml("../configs/datapaths.yaml")

In [11]:
data

{'NORMPARAMS': {'chm': {'_001': {'mu': 29.136810302734375,
    'n': 424127,
    'std': 5.059319972991943},
   '_010': {'mu': 27.330501556396484, 'n': 331027, 'std': 6.319406986236572},
   '_011': {'mu': 28.345002320026218, 'n': 755154, 'std': 5.611687865810269},
   '_100': {'mu': 29.79595947265625, 'n': 753596, 'std': 9.429980278015137},
   '_101': {'mu': 29.5585836550901, 'n': 1177723, 'std': 7.85599807406007},
   '_110': {'mu': 29.043501579318452, 'n': 1084623, 'std': 8.480633136144107},
   '_111': {'mu': 29.069731736028462, 'n': 1508750, 'std': 7.518862605606749}},
  'info': {'description': 'CHM normalization parameters',
   'encoding': '001: FirstSite only, 010: Second Site, 100: Third Site',
   'version': '1.0'}},
 'SITE1': {'CHM': '../data/01_input_pipeline/01_Ebrach/01_CHM.tif',
  'CHM_norm': '../data/01_input_pipeline/01_Ebrach/01_CHM_norm111.tif',
  'DEM': '../data/01_input_pipeline/01_Ebrach/01_DEM10.tif',
  'DLT': '../data/01_input_pipeline/01_Ebrach/01_DLT_P.tif',
  'FMASK'

In [13]:
array, desc = bsc.read_multiband_tif_as_stack(data['SITE1']['S2_norm']['spring'])

In [14]:
array.shape

(13, 5, 846, 1241)

In [15]:
desc

('BLU_Q25',
 'BLU_Q50',
 'BLU_Q75',
 'BLU_AVG',
 'BLU_STD',
 'BNR_Q25',
 'BNR_Q50',
 'BNR_Q75',
 'BNR_AVG',
 'BNR_STD',
 'EVI_Q25',
 'EVI_Q50',
 'EVI_Q75',
 'EVI_AVG',
 'EVI_STD',
 'GRN_Q25',
 'GRN_Q50',
 'GRN_Q75',
 'GRN_AVG',
 'GRN_STD',
 'NBR_Q25',
 'NBR_Q50',
 'NBR_Q75',
 'NBR_AVG',
 'NBR_STD',
 'NDV_Q25',
 'NDV_Q50',
 'NDV_Q75',
 'NDV_AVG',
 'NDV_STD',
 'NIR_Q25',
 'NIR_Q50',
 'NIR_Q75',
 'NIR_AVG',
 'NIR_STD',
 'RE1_Q25',
 'RE1_Q50',
 'RE1_Q75',
 'RE1_AVG',
 'RE1_STD',
 'RE2_Q25',
 'RE2_Q50',
 'RE2_Q75',
 'RE2_AVG',
 'RE2_STD',
 'RE3_Q25',
 'RE3_Q50',
 'RE3_Q75',
 'RE3_AVG',
 'RE3_STD',
 'RED_Q25',
 'RED_Q50',
 'RED_Q75',
 'RED_AVG',
 'RED_STD',
 'SW1_Q25',
 'SW1_Q50',
 'SW1_Q75',
 'SW1_AVG',
 'SW1_STD',
 'SW2_Q25',
 'SW2_Q50',
 'SW2_Q75',
 'SW2_AVG',
 'SW2_STD')

In [25]:
def load_raster(path):
    with rasterio.open(path) as src:
        arr = src.read().astype(np.float32)  # shape (bands, H, W)
        if src.nodata is not None:
            arr[arr == src.nodata] = np.nan
    return arr

In [ ]:
QUANTILE_IDX = {"Q25": 0, "Q50": 1, "Q75": 2, "AVG": 3, "STD": 4}
stack = []
seasons = ["summer", "spring"]
quantiles = ["Q50", "Q75"]

for season in seasons:
    arr, desc = bsc.read_multiband_tif_as_stack(data["SITE1"]["S2"][season])  # (bands, channels, H, W)
    arr = arr[:, [QUANTILE_IDX[q] for q in quantiles], :, :]                  # select quantiles
    arr = arr.reshape(-1, arr.shape[-2], arr.shape[-1])                       # (bands*quantiles, H, W)
    stack.append(arr)
    print(f"Season {season} shape:", arr.shape)

# Aux inputs
auxes = ['DLT', 'DEM']
for aux in auxes:
    arr = load_raster(data["SITE1"][aux])                                     # (1, H, W)
    stack.append(arr)

X = np.concatenate(stack, axis=0)                                             # (C, H, W)
print("Final stack shape:", X.shape)


Season summer shape: (26, 846, 1241)
Season spring shape: (26, 846, 1241)
Final stack shape: (54, 846, 1241)


In [24]:
X.shape

(4, 13, 846, 1241)

In [28]:
auxtest = bsc.read_tif_as_array(data["SITE1"]["DLT"])

✅ Loaded 01_DLT_P.tif: shape=(846, 1241), CRS=EPSG:25832, GSD=10.00 x 9.99 metre
Band names: (None,)


In [65]:
from src.config_loader import get_config, load_yaml
from src.data_loader import*

def main():
    cfg = get_config("baseline")  # experiment config (site arg irrelevant if we loop sites)

    sites = load_yaml("../configs/datapaths.yaml")

    # Build dataset from all sites
    X, Y = build_patched_dataset(cfg, sites, patch_size=32)

    print("Total patches:", len(X))
    print("One patch input shape:", X[0].shape)
    print("One patch outputs:", {k: v.shape for k,v in Y[0].items()})

In [96]:
sites, cfg = get_config("baseline")  # experiment config (site arg irrelevant if we loop sites)

# sites = load_yaml("../configs/datapaths.yaml")

# for site_name, site_paths in sites.items():
#     X, Y = build_site_data(cfg, site_paths)
    

# Build dataset from all sites
X, Y = build_patched_dataset(cfg, sites, patch_size=32)

#print("One patch outputs:", {k: v.shape for k,v in Y[0].items()})

In [97]:
X.shape

(1172, 14, 32, 32)

In [98]:
Y.shape

(1172, 1, 32, 32)

In [ ]:
print("mean NaN percentage in X patches:")
print((np.isnan(X).mean(axis=(1, 2, 3)) * 100).mean())

max NaN percentage in X patches:
1.0126419848549488


In [ ]:
print("mean NaN percentage in Y patches:")
print((np.isnan(Y).mean(axis=(1, 2, 3)) * 100).mean())

max NaN percentage in Y patches:
0.8822392278156996


In [82]:
cfg

{'spectral': {'seasons': ['summer'], 'quantiles': ['Q50']},
 'aux_inputs': ['FMASK'],
 'strategy': 'all_in_model',
 'outputs': {'canopy_height': {'target': 'CHM_norm', 'loss': 'huber'}},
 'exp': 'baseline'}

In [71]:
sites

{'NORMPARAMS': {'chm': {'_001': {'mu': 29.136810302734375,
    'n': 424127,
    'std': 5.059319972991943},
   '_010': {'mu': 27.330501556396484, 'n': 331027, 'std': 6.319406986236572},
   '_011': {'mu': 28.345002320026218, 'n': 755154, 'std': 5.611687865810269},
   '_100': {'mu': 29.79595947265625, 'n': 753596, 'std': 9.429980278015137},
   '_101': {'mu': 29.5585836550901, 'n': 1177723, 'std': 7.85599807406007},
   '_110': {'mu': 29.043501579318452, 'n': 1084623, 'std': 8.480633136144107},
   '_111': {'mu': 29.069731736028462, 'n': 1508750, 'std': 7.518862605606749}},
  'info': {'description': 'CHM normalization parameters',
   'encoding': '001: FirstSite only, 010: Second Site, 100: Third Site',
   'version': '1.0'}},
 'SITE1': {'CHM': '../data/01_input_pipeline/01_Ebrach/01_CHM.tif',
  'CHM_norm': '../data/01_input_pipeline/01_Ebrach/01_CHM_norm111.tif',
  'DEM': '../data/01_input_pipeline/01_Ebrach/01_DEM10.tif',
  'DLT': '../data/01_input_pipeline/01_Ebrach/01_DLT_P.tif',
  'FMASK'